For my hypothesis test, I am hypothesizing that various columns will have adverse effects on whether or not someone will have diabetes. I am going to run *** seperate tests in order to test the if the values in the *** columns can indicate whether someone is more or less likely to get diabetes.

H0: The mean of the values in *** columns will be roughly the same for patients with diabetes and those without.

H1: The mean of the values in the *** columns will be significantly different for the patients with diabetes compared to patients without diabetes.


**Results**
According the the p-values, BMI, physical health and mental health are all strong indicators of a patients alikelihood to end up with diabetes. All of the columns showed a p-value of 0 which indicates that there is very significant differences in their values for patients with diabetes compared to those without. Upon further analysis, it may appear that columns such as the mental health columns might not directly effect whether or not a patient has diabetes. Instead, it is likely that mental health is conditionally indepedent from diabetes, given other factors such as physical health or BMI.

In [7]:
import pandas as pd
import scipy.stats as stats

df = pd.read_csv('diabetes.csv')

categorical_vars = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack',
                    'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare',
                    'NoDocbcCost', 'DiffWalk', 'Sex']

chi_results = {}

for var in categorical_vars:
    contingency_table = pd.crosstab(df[var], df['Diabetes_binary'])
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    chi_results[var] = p

continuous_vars = ['BMI', 'MentHlth', 'PhysHlth']

t_results = {}

for var in continuous_vars:
    group1 = df[df['Diabetes_binary'] == 0][var]
    group2 = df[df['Diabetes_binary'] == 1][var]
    t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=True, permutations=1000, alternative='less')
    t_results[var] = p_val

print("Chi-Square Test Results:")
for var, p_val in chi_results.items():
    print(f"{var}: p-value = {p_val}")

print("\nT-Test Results:")
for var, p_val in t_results.items():
    print(f"{var}: p-value = {p_val}")


Chi-Square Test Results:
HighBP: p-value = 0.0
HighChol: p-value = 0.0
CholCheck: p-value = 3.7513988789188705e-233
Smoker: p-value = 8.640171809942516e-206
Stroke: p-value = 0.0
HeartDiseaseorAttack: p-value = 0.0
PhysActivity: p-value = 0.0
Fruits: p-value = 1.0881213885155634e-93
Veggies: p-value = 1.4630291539062376e-178
HvyAlcoholConsump: p-value = 1.8659318101056406e-181
AnyHealthcare: p-value = 2.986181073970452e-16
NoDocbcCost: p-value = 2.218395410122378e-56
DiffWalk: p-value = 0.0
Sex: p-value = 2.1098747365768505e-56

T-Test Results:
BMI: p-value = 0.0
MentHlth: p-value = 0.0
PhysHlth: p-value = 0.0


In [3]:
df

,Unnamed: 0,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,0,1,1,1,40,1,0,0,0,0,...,0,5,18,15,1,0,9,4,3,0
1,1,0,0,0,25,1,0,0,1,0,...,1,3,0,0,0,0,7,6,1,0
2,2,1,1,1,28,0,0,0,0,1,...,1,5,30,30,1,0,9,4,8,0
3,3,1,0,1,27,0,0,0,1,1,...,0,2,0,0,0,0,11,3,6,0
4,4,1,1,1,24,0,0,0,1,1,...,0,2,3,0,0,0,11,5,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,253675,1,1,1,45,0,0,0,0,1,...,0,3,0,5,0,1,5,6,7,0
253676,253676,1,1,1,18,0,0,0,0,0,...,0,4,0,0,1,0,11,2,4,1
253677,253677,0,0,1,28,0,0,0,1,1,...,0,1,0,0,0,0,2,5,2,0
253678,253678,1,0,1,23,0,0,0,0,1,...,0,3,0,0,0,1,7,5,1,0
